# Verification problem

Verification of transient solution of 1D rod from the [NAFEMS](https://www.nafems.org) test suite, see section "1D Single Equation" on [wolfram.com](https://reference.wolfram.com/language/PDEModels/tutorial/HeatTransfer/HeatTransferVerificationTests.html).

In [31]:
using MMJMesh
using MMJMesh.Plots
using MMJMesh.Meshes
using MMJMesh.Utilities
using MMJMesh.Geometries

import Plots
import CairoMakie
using DifferentialEquations

include("fem.jl")
include("heat.jl")
CairoMakie.update_theme!(colormap=:acton)

## Mesh

In [ ]:
m = Mesh("gmsh/rod.msh")
mplot(m, edgesvisible=true) |> mconf()

## Element functions

In [33]:
setdata!(group(m, :elements), :ke_func, heat_ke(35))
setdata!(group(m, :elements), :me_func, heat_me(7200, 440.5))

## Boundary conditions

We use Robin BCs in order to impose Dirichlet BCs using a large value for the heat transfer coefficient  als penalty parameter.

In [34]:
pen = 1e10
setdata!(group(m, :bl), :ke_func, robin_ke(pen))
setdata!(group(m, :bl), :re_func, robin_re(pen, 1))
setdata!(group(m, :br), :ke_func, robin_ke(pen))

## Initial condition

In [35]:
t0 = zeros(nnodes(m));

### Solve initial value problem

Based on: https://docs.sciml.ai/DiffEqDocs/stable/tutorials/faster_ode_example/

#### ODE function

In [43]:
K, M, r = assemble_kmr(m)
F(ΘHat, _, t) = -K * ΘHat + 100 * sin(pi * t / 40) * r
JF(_, _, _) = -K
FM = ODEFunction(F, mass_matrix=M, jac=JF);

#### Solve initial value problem

In [ ]:
p = ODEProblem(FM, t0, (0, 32))
@time s = solve(p, QNDF())
Plots.plot(s, legend=false)

Use Euler's method (just to try it, not accurate)

In [ ]:
pen = 1e4
setdata!(group(m, :bl), :ke_func, robin_ke(pen))
setdata!(group(m, :bl), :re_func, robin_re(pen, 1))
setdata!(group(m, :br), :ke_func, robin_ke(pen))

K, M, r = assemble_kmr(m)

luM = lu(M)
F(ΘHat, _, t) =  luM \ (-K * ΘHat + 100 * sin(pi * t / 40) * r)

p = ODEProblem(F, t0, (0.0, 32.0))
@time s2 = solve(p, Euler(), dt=0.01)
Plots.plot(s2, legend=false)

## Plot results

In [ ]:
mplot(m, s.u[end]) |> mconf()

## Compare to reference value

The relative error should be smaller than one permille.

In [ ]:
tref = 36.6
tact = s.u[end][nodeindex(m, on(Point(0.02, 0.005)))]
error = abs(tref - tact) / tref

# Print results
println("   Temperature: ", tact)
println("Relative error: ", error)
println("   Test passed: ", error < 1e-3)
